In [ ]:
import os
import numpy as np
import mne
import imageio
import h5py
# import scipy.fftpack
import matplotlib
import pywt
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
# from scipy import signal
from matplotlib.colors import ListedColormap
import time
import seaborn as sns
# import scipy.io as sio
# from scipy.integrate import simps
import pandas as pd
# from scipy import fft
import matplotlib.mlab as mlab
import sys

sys.path.append('T:\EL_experiment\Codes\CCEP_human\Python_Analysis/py_functions')
import analys_func
from scipy.stats import norm
import LL_funcs
from scipy.stats import norm
from tkinter import filedialog
from tkinter import *
import ntpath

root = Tk()
root.withdraw()
import math
import scipy
from scipy import signal
import pylab
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform
import platform
from glob import glob
from scipy.io import savemat
import scipy.cluster.hierarchy as spc
from scipy.spatial import distance

import basic_func as bf
from scipy.integrate import simps
from numpy import trapz
import IO_func as IOF
import BM_func as BMf
import tqdm
from matplotlib.patches import Rectangle
from pathlib import Path

import freq_funcs as ff

regions = pd.read_excel("T:\EL_experiment\Patients\\" + 'all' + "\elab_labels.xlsx", sheet_name='regions', header=0)
color_regions = regions.color.values
regions = regions.label.values

CR_color = pd.read_excel("T:\EL_experiment\Patients\\" + 'all' + "\Analysis\BrainMapping\CR_color.xlsx", header=0)
CR_color_a = CR_color.a.values
CR_color = CR_color.c.values
CR_color = np.zeros((24, 3))
CR_color[6:18, :] = np.array([253, 184, 19]) / 255

dist_groups = np.array([[0, 30], [30, 60], [60, 120]])
dist_labels = ['local (<30 mm)', 'short (<60mm)', 'long']
Fs = 500
dur = np.zeros((1, 2), dtype=np.int32)
t0 = 1
dur[0, 0] = -t0
dur[0, 1] = 3

folder = 'BrainMapping'
# dur[0,:]       = np.int32(np.sum(abs(dur)))
x_ax = np.arange(dur[0, 0], dur[0, 1], (1 / Fs))
color_elab = np.zeros((3, 3))
color_elab[0, :] = np.array([31, 78, 121]) / 255
color_elab[1, :] = np.array([189, 215, 238]) / 255
color_elab[2, :] = np.array([0.256, 0.574, 0.431])
cwd = os.getcwd()

In [ ]:
subj = 'EL011'
cond_folder = 'CR'  # Condition = 'Hour', 'Condition', 'Ph'

if cond_folder == 'Ph':
    cond_vals = np.arange(4)
    cond_labels = ['BM', 'BL', 'Fuma', 'BZD']
    cond_colors = ['#494159', '#594157', "#F1BF98", "#8FB996"]
    cond1 = 'Condition'  # 'condition', 'h'
    cond_folder = 'Ph'  # 'Ph', 'Sleep', 'CR'
    Condition = 'Condition'
if cond_folder == 'CR':
    Condition = 'Hour'  # Condition = 'Hour'
    cond1 = 'h'  # h (as stored in stimlist)

######## General Infos

path_patient_analysis = 'y:\\eLab\EvM\Projects\EL_experiment\Analysis\Patients\\' + subj
path_gen = os.path.join('y:\\eLab\Patients\\' + subj)
if not os.path.exists(path_gen):
    path_gen = 'T:\\EL_experiment\\Patients\\' + subj
path_patient = path_gen + '\Data\EL_experiment'  # os.path.dirname(os.path.dirname(cwd))+'/Patients/'+subj
path_infos = os.path.join(path_patient, 'infos')
if not os.path.exists(path_infos):
    path_infos = path_gen + '\\infos'
    
sep = ';'
Fs = 500
Path(path_patient_analysis + '\\' + folder + '/' + cond_folder + '/data').mkdir(parents=True, exist_ok=True)
Path(path_patient_analysis + '\\' + folder + '/' + cond_folder + '/BM_plot_trial').mkdir(parents=True, exist_ok=True)
Path(path_patient_analysis + '\\' + folder + '/' + cond_folder + '/figures/single_con').mkdir(parents=True, exist_ok=True)


# get labels
if cond_folder == 'Ph':
    files_list = glob(path_patient_analysis + '\\' + folder + '/data/Stim_list_*Ph*')
else:
    files_list = glob(path_patient_analysis + '\\' + folder + '/data/Stim_list_*')
i = 0
stimlist = pd.read_csv(files_list[i])
# EEG_resp = np.load(path_patient + '/Analysis/' + folder + '/data/ALL_resps_'+files_list[i][-11:-4]+'.npy')
lbls = pd.read_excel(os.path.join(path_infos, subj + "_labels.xlsx"), header=0, sheet_name='BP')
labels_all, labels_region, labels_clinic, coord_all, StimChans, StimChanSM, StimChansC, StimChanIx, stimlist = bf.get_Stim_chans(
    stimlist,
    lbls)
badchans = pd.read_csv(path_patient_analysis + '\\' + folder + '/data/badchan.csv')
bad_chans = np.unique(np.array(np.where(badchans.values[:, 1] == 1))[0, :])

bad_region = np.where((labels_region == 'WM') | (labels_region == 'OUT') | (labels_region == 'Putamen'))[0]

file_con = path_patient_analysis + '\\' + folder + '/' + cond_folder + '/data/con_trial_all.csv'
######### Load data
rerun = 0
if os.path.isfile(file_con):
    # con_trial
    con_trial = pd.read_csv(file_con)

## Loading Data

In [ ]:
sig_trial_all = np.load(path_patient + '/Analysis/BrainMapping/' + cond_folder + '/surrogate/sig_trial_all.npy')

In [ ]:
sig_mean_all = np.load(path_patient + '/Analysis/BrainMapping/' + cond_folder + '/surrogate/sig_trial_mean.npy')

In [ ]:
ts_surr_all = np.load(path_patient + '/Analysis/BrainMapping/' + cond_folder + '/surrogate/sig_ts_mean.npy')

In [ ]:
arr = np.zeros((1,8))
for b in range(len(sig_mean_all)):
    arr_new = np.zeros((len(labels_all),8))
    arr_new[:,5] = np.arange(len(labels_all))
    arr_new[:,0] = b
    # trial
    LL = np.nanpercentile(sig_trial_all[b,:,:,0],90, axis=1)
    P2P = np.nanpercentile(sig_trial_all[b,:,:,1],90, axis=1)
    arr_new[:,1] = LL
    arr_new[:,2] = P2P
    # mean 
    LL = np.nanpercentile(sig_mean_all[b,:,:,0],99, axis=1)
    P2P = np.nanpercentile(sig_mean_all[b,:,:,1],99, axis=1)
    arr_new[:,3] = LL
    arr_new[:,4] = P2P
    # # ts_surr_all
    arr_new[:,6] = np.nanpercentile(ts_surr_all[b,:,:,0],90, axis=1)
    arr_new[:,7] = np.nanpercentile(ts_surr_all[b,:,:,1],90, axis=1)
    arr = np.concatenate([arr,arr_new],0)

In [ ]:
surr_data = pd.DataFrame(arr, columns=['Block', 'LL_trial', 'P2P_trial', 'LL_mean', 'P2P_mean', 'Chan', 'Pearson', 'Phase'])

In [ ]:
sns.catplot(x='Block', y='P2P_trial', data =surr_data, aspect= 3)

In [ ]:
sns.catplot(x='Block', y='Phase', data =surr_data, aspect= 3)

## brain plots and correlation

In [ ]:
non_stim = np.arange(len(labels_all))
non_stim = np.delete(non_stim, StimChanIx, 0)
bad_all = np.unique(np.concatenate([bad_region, bad_chans, non_stim])).astype('int')

In [ ]:
labels_region_L = lbls.Hemisphere.values+'_'+labels_region

In [ ]:
files_list = glob(path_patient + '/Analysis/' + folder + '/data/Stim_list_*CR*')

In [ ]:
sig_trial_label = ['LL', 'P2P']

In [ ]:
l = 0
print('loading ' + files_list[l][-11:-4], end='\r')
stimlist0 = pd.read_csv(files_list[l])
EEG_resp0 = np.load(
    path_patient + '/Analysis/' + folder + '/data/ALL_resps_' + files_list[l][-11:-4] + '.npy')
block_l = files_list[l][-11:-4]
file = path_patient + '/Analysis/BrainMapping/' + cond_folder + '/data/con_trial_' + block_l + '.csv'
con_trial_block0 = pd.read_csv(file)

In [ ]:
l = 8
print('loading ' + files_list[l][-11:-4], end='\r')
stimlist8 = pd.read_csv(files_list[l])
EEG_resp8 = np.load(
    path_patient + '/Analysis/' + folder + '/data/ALL_resps_' + files_list[l][-11:-4] + '.npy')
block_l = files_list[l][-11:-4]
file = path_patient + '/Analysis/BrainMapping/' + cond_folder + '/data/con_trial_' + block_l + '.csv'
con_trial_block8 = pd.read_csv(file)

## plot functions

In [ ]:
from scipy.signal import hilbert, butter, filtfilt
from scipy.fftpack import fft,fftfreq,rfft,irfft,ifft
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats as stats
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y


In [ ]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))

In [ ]:
def get_cross_corr(x, y, Fs=500, win=0.25):
    s0 = 0
    s1 = int(s0+win*Fs)
    corr = np.zeros_like(x)
    i = int(win/2*Fs)
    while s1<x.shape[-1]:
        al1 = np.angle(hilbert(x[s0:s1]),deg=False)
        al2 = np.angle(hilbert(y[s0:s1]),deg=False)
        corr[i] = crosscorr(x[s0:s1], y[s0:s1])# np.mean(1-np.sin(np.abs(al1-al2)/2))
        s0 = s0+1
        s1 = s1+1
        i = i+1
    return corr

In [ ]:
def get_phase_sync(x, y, Fs=500, win=0.25):
    s0 = 0
    s1 = int(s0+win*Fs)
    corr = np.zeros_like(x)
    i = int(win/2*Fs)
    while s1<x.shape[-1]:
        al1 = np.angle(hilbert(x[s0:s1]),deg=False)
        al2 = np.angle(hilbert(y[s0:s1]),deg=False)
        corr[i] = np.mean(1-np.sin(np.abs(al1-al2)/2))
        s0 = s0+1
        s1 = s1+1
        i = i+1
    return corr
def get_pearson_ts(x, y, Fs, win= 0.25):
    
    s0 = 0
    s1 = int(s0+win*Fs)
    corr = np.zeros_like(x)
    i = int(win/2*Fs)
    while s1<x.shape[-1]:
        corr[i] = np.corrcoef(x[s0:s1], y[s0:s1])[0,1]
        s0 = s0+1
        s1 = s1+1
        i = i+1
    return corr

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [ ]:
color_elab      = np.zeros((3,3))
color_elab[0,:] = np.array([31, 78, 121])/255
color_elab[1,:] = np.array([189, 215, 238])/255
color_elab[2,:] = np.array([0.256, 0.574, 0.431])

In [ ]:
def get_dtw_ts(x, y, Fs, ov=0.1, win= 0.25):
    corr = np.zeros((int(len(x)/(win*Fs*ov)),))
    i = 0#int(win/2*Fs)
    x = np.pad(x, (int(0.5*win*Fs), int(0.5*win*Fs)), 'constant', constant_values=(0,0))
    y = np.pad(y, (int(0.5*win*Fs), int(0.5*win*Fs)), 'constant', constant_values=(0,0))
    s0 = 0
    s1 = int(s0+win*Fs)
    
    while (i<len(corr))&(s1<x.shape[-1]):
        corr[i], _ = fastdtw(x[s0:s1],y[s0:s1], dist=euclidean)
        s0 = s0+int(ov*win*Fs)
        s1 = s1+int(ov*win*Fs)
        i = i+1
    return corr

In [ ]:
sc = 40
rc = 43
lists  = con_trial_block[(con_trial_block['Chan']==rc)&(con_trial_block['Stim']==sc)&(~np.isnan(con_trial_block.LL.values))] #
EEG_trial  = EEG_resp[rc,lists.Num_block.values.astype('int'),: ]

In [ ]:
con_trial_block0.LL.values

In [ ]:
ts_surr_all.shape

In [ ]:
np.nanpercentile(ts_surr_all[8,rc,:,0],90)

In [ ]:
con_trial_block8

In [ ]:
con_trial_block8[(con_trial_block8.Stim==sc)&(con_trial_block8.Chan==rc)]

In [ ]:
sc = 4
rc = 42

plot_surr(sc,rc, EEG_resp8, con_trial_block8,sig_trial_all[8],sig_mean_all[8], ts_surr_all[8],w=0.5)

In [ ]:
import itertools

In [ ]:
sig_trial_all

In [ ]:
sig_mean_all

In [ ]:
comb

In [ ]:
comb = np.array(list(itertools.combinations(range(3), 2)))

In [ ]:
def plot_surr(sc,rc, EEG_resp, con_trial_block,sig_trial, sig_mean, ts_surr, w=0.5):
    lists  = con_trial_block[(con_trial_block['Chan']==rc)&(con_trial_block['Stim']==sc)] #
    #&(~np.isnan(con_trial_block.LL.values))
    EEG_trial  = EEG_resp[rc,lists.Num_block.values.astype('int'),: ]
    
    LL_trial = np.round(np.nanpercentile(sig_trial[rc,:,0],90),2)
    LL_mean = np.round(np.nanpercentile(sig_mean[rc,:,0],99),2)
    fig = plt.figure(figsize=(8,15))
    plt.suptitle(labels_all[sc]+ ' -- '+labels_all[rc], fontsize=35)
    gs       = fig.add_gridspec(3, 1)  # GridSpec(4,1, height_ratios=[1,2,1,2])
    fig.add_subplot(gs[0,0])
    for i in range(len(lists)):
        plt.plot(x_ax, ff.lp_filter(EEG_trial[i], 45, Fs), color=color_elab[0],label=str(i+1)+'. LL '+str(np.round(lists.LL_peak.values[i],2))+'/'+str(LL_trial))
    plt.plot(x_ax, ff.lp_filter(np.mean(EEG_trial,0), 45, Fs), color=[0,0,0], linewidth=3,label='LL '+str(np.round(lists.LL_mean.values[0],2))+'/'+str(LL_mean)) 
    plt.axvline(0, color = [0,0,0])
    #plt.ylim([-600,1000])
    plt.legend()
    # pearson
    
    fig.add_subplot(gs[1,0])
    plt.title('Pearson Correlation, win = '+str(w)+'s')
    comb = np.array(list(itertools.combinations(range(len(EEG_trial)), 2)))
    for i in range(len(comb)):
        corr_xy = get_pearson_ts(EEG_trial[comb[i,0]],EEG_trial[comb[i,1]], Fs, win= w)
        plt.plot(x_ax, corr_xy, color=[0,0,0], label=str(comb[i,0]+1)+'-'+str(comb[i,1]+1)+'. max: '+str(np.round(np.max(corr_xy[500:750]),2)))
    plt.ylim([-1.05,1.05])
    plt.axhline(np.nanpercentile(ts_surr[rc,:,0],90), color= [1,0,0])
    plt.axvline(0, color = [0,0,0])
    plt.legend()
    
    # Phase ynchronization
    fig.add_subplot(gs[2,0])
    plt.title('Pearson Correlation, win = '+str(w)+'s')
    comb = np.array(list(itertools.combinations(range(len(EEG_trial)), 2)))
    for i in range(len(comb)):
        #corr_xy = get_pearson_ts(EEG_trial[comb[i,0]],EEG_trial[comb[i,1]], Fs, win= w)
        corr_xy = get_phase_sync(ff.bp_filter(EEG_trial[comb[i,0]],1,20,Fs),ff.bp_filter(EEG_trial[comb[i,1]],1,20,Fs), Fs, win= w)
        plt.plot(x_ax, corr_xy, color=[0,0,0], label=str(comb[i,0]+1)+'-'+str(comb[i,1]+1)+'. max: '+str(np.round(np.max(corr_xy[625:750]),2)))
    plt.ylim([0,1.1])
    plt.axhline(np.nanpercentile(ts_surr[rc,:,1],90), color= [1,0,0])
    plt.axvline(0, color = [0,0,0])
    plt.legend()
    plt.xlabel('Time [s]', fontsize=25)
    
# fig.add_subplot(gs[3,0])
# # plt.title('DTW, win = '+str(w)+'s')
# plt.title('Phase Synch, win = '+str(w)+'s')
# plt.plot(x_ax, ps_xy, color=color_elab[0])
# plt.ylim([0,1])


In [ ]:
w = 0.5
x = EEG_trial[0]
y = EEG_trial[1]
corr_xy = get_pearson_ts(x,y, Fs, win= w)
ps_xy = get_phase_sync(ff.bp_filter(EEG_plot[0],1,20,Fs),ff.bp_filter(EEG_plot[1],2,20,Fs), Fs, win= w)
dtw_xy = get_dtw_ts(x,y, Fs, ov=0.25, win= w)
x_ax_dtw = np.linspace(x_ax[0], x_ax[-1], num = len(dtw_xy), endpoint=True)
EEG_plot = np.stack([x,y],0)
fig = plt.figure(figsize=(10,20))
plt.suptitle(labels_all[sc]+ ' -- '+labels_all[rc], fontsize=35)
gs       = fig.add_gridspec(5, 1)  # GridSpec(4,1, height_ratios=[1,2,1,2])
for i in range(2):
    fig.add_subplot(gs[i,0])
    plt.plot(x_ax, EEG_plot[i], color=[0,0,0],label='Raw')
    plt.plot(x_ax, ff.bp_filter(EEG_plot[i],1,20,Fs), color=color_elab[0], label='BP filter 1-20Hz')
    plt.axvline(0, color = [0,0,0])
    plt.ylim([-600,1000])
    plt.legend()
fig.add_subplot(gs[2,0])
plt.title('Pearson Correlation, win = '+str(w)+'s')
plt.plot(x_ax, corr_xy, color=[0,0,0])
plt.ylim([-1,1])

fig.add_subplot(gs[3,0])
# plt.title('DTW, win = '+str(w)+'s')
plt.title('Phase Synch, win = '+str(w)+'s')
plt.plot(x_ax, ps_xy, color=color_elab[0])
plt.ylim([0,1])
fig.add_subplot(gs[4,0])
plt.title('DTW, win = '+str(w)+'s')
plt.plot(x_ax_dtw, dtw_xy, color=[0,0,0])
#plt.ylim([-1,1])

plt.xlabel('time [s]', fontsize=20)
plt.tight_layout()


In [ ]:
for i in range(2):
    plt.plot(x_ax, EEG_trial[i])

plt.plot(x_ax, corr_xy*300)

plt.axvline(0, color = [0,0,0])

In [ ]:
def plot_mean(sc, rc, LL_CCEP,EEG_resp, labels ):
    t_0    = 1
    lists  = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==sc)&(~np.isnan(LL_CCEP.LL.values))] #
    h = np.unique(lists.Hour)[0].astype('int')
    b = np.unique(lists.Block)[0].astype('int')
    fig   = plt.figure(figsize=(12,7) )
    #plt.title(labels_all[Stim_chs]+' -- '+labels_clinic[rc])
    #plt.title(labels[sc]+' -- '+labels[rc])
    plt.title(labels[sc]+' -- '+labels[rc]+', Dist: '+str(np.round(lists.d.values[0]))+'mm, '+f"{h:02}"+":00")
    
    plt.axvline(0, c=[0,0,0])
    plt.xlabel('time [s]')
    #plt.text(-0.37, 200, 'LL: '+str(np.round(np.mean(lists.LLpeak),2))+'uV/ms (of mean)', c=[0,0,0])
    
    ylim = 1000

    #stim_spec                = stimlist[(stimlist.condition>0)&(stimlist.ChanP==ChanP)&(stimlist.IPI_ms ==0)&(stimlist.noise ==0)]#&(stimlist.noise ==0)
    stimNum_all                  = lists.Num_block.values.astype('int')
    for i in range(len(stimNum_all)):
        ylim =np.max([ylim, np.max(abs(ff.lp_filter(EEG_resp[rc,stimNum_all[i],Fs:int(1.5*Fs)],45,Fs)))])
        plt.plot(x_ax,ff.lp_filter(EEG_resp[rc,stimNum_all[i],:],45,Fs), c=color_elab[0], linewidth=1)
    mn = ff.lp_filter(np.mean(EEG_resp[rc,stimNum_all,:],0),45,Fs)
    st  = np.std(ff.lp_filter(EEG_resp[rc,stimNum_all,:],45,Fs),0)
    plt.plot(x_ax,mn, c=[0,0,0], linewidth=3, label='mean, n='+str(len(stimNum_all)))
    plt.fill_between(x_ax, mn-st, mn+st, color=color_elab[0], alpha= 0.3)
    plt.xlim([-0.6,1])
    plt.ylim([-np.max([ylim*1.071,300]),np.max([ylim*1.071,300])])
    #plt.axvspan(t_0+w_r-1, t_0-1+w_LL, alpha=0.3, color=color_elab[0])
    #plt.axvspan(t_0+0.015-1, t_0+w_r-1, alpha=0.8, color=color_elab[1])
    #plt.axvspan(t_0+0.015-1-0.2, t_0+w_r-1-0.2, alpha=0.8, color=color_elab[1])
    plt.ylabel('[\u03BCV]', fontsize=18) #\u0394 delta
    plt.xlabel('time [s]', fontsize=18)
    plt.legend()
    plt.savefig(path_patient + '/Analysis/BrainMapping/'+cond_folder+'/figures/single_con/BM_'+str(b)+'_'+labels_all[sc]+'_'+labels_all[rc]+'.svg')
    plt.savefig(path_patient + '/Analysis/BrainMapping/'+cond_folder+'/figures/single_con/BM_'+str(b)+'_'+labels_all[sc]+'_'+labels_all[rc]+'.jpg')
    plt.show()

In [ ]:
def plot_mean_zscore(sc, rc, LL_CCEP,EEG_resp, labels ):
    t_0    = 1
    lists  = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==sc)&(~np.isnan(LL_CCEP.LL.values))] #&(~np.isnan(LL_CCEP.LL.values))
    h = np.unique(lists.Hour)[0].astype('int')
    b = np.unique(lists.Block)[0].astype('int')
    fig   = plt.figure(figsize=(12,7) )
    #plt.title(labels_all[Stim_chs]+' -- '+labels_clinic[rc])
    #plt.title(labels[sc]+' -- '+labels[rc])
    plt.title(labels[sc]+' -- '+labels[rc]+', Dist: '+str(np.round(lists.d.values[0]))+'mm, '+f"{h:02}"+":00")
    
    plt.axvline(0, c=[0,0,0])
    plt.xlabel('time [s]')
    #plt.text(-0.37, 200, 'LL: '+str(np.round(np.mean(lists.LLpeak),2))+'uV/ms (of mean)', c=[0,0,0])
    
    ylim = 1000

    #stim_spec                = stimlist[(stimlist.condition>0)&(stimlist.ChanP==ChanP)&(stimlist.IPI_ms ==0)&(stimlist.noise ==0)]#&(stimlist.noise ==0)
    stimNum_all                  = lists.Num_block.values.astype('int')
    for i in range(len(stimNum_all)):
        ylim =np.max([ylim, np.max(abs(ff.lp_filter(EEG_resp[rc,stimNum_all[i],Fs:int(1.5*Fs)],45,Fs)))])
        resp = ff.lp_filter(bf.zscore_CCEP(EEG_resp[rc,stimNum_all[i],:]),45,Fs)
        plt.plot(x_ax,resp, c=color_elab[0], linewidth=1, label='trial'+str(i+1))
    plt.plot(x_ax,ff.lp_filter(np.mean(bf.zscore_CCEP(EEG_resp[rc,stimNum_all,:]),0),45,Fs), c=[0,0,0], linewidth=2, label='mean, n='+str(len(stimNum_all)))
    plt.plot(x_ax,bf.zscore_CCEP(ff.lp_filter(np.mean(EEG_resp[rc,stimNum_all,:],0),45,Fs)), c=[1,0,0], linewidth=2, label='mean, n='+str(len(stimNum_all)))

    plt.xlim([-0.5,1])
    plt.ylim([-6,6])
    #plt.axvspan(t_0+w_r-1, t_0-1+w_LL, alpha=0.3, color=color_elab[0])
    #plt.axvspan(t_0+0.015-1, t_0+w_r-1, alpha=0.8, color=color_elab[1])
    #plt.axvspan(t_0+0.015-1-0.2, t_0+w_r-1-0.2, alpha=0.8, color=color_elab[1])
    plt.legend()
    #plt.savefig()
    plt.show()